In [ ]:
import numpy as np
import pandas as pd
import sklearn 
from sklearn.model_selection import train_test_split
import matplotlib as plt
import xgboost as xgb

In [ ]:
df = pd.read_csv(r"C:\Users\Asus\Downloads\matches.csv")
df
col = df['result'].replace({'W':0,'D':1,'L':2})
df.pop('result')

df['result'] = col
df.columns

In [ ]:
df['team'].value_counts()

ENCODING CATEGORICAL COLUMNS

In [ ]:
from sklearn.preprocessing import LabelEncoder

# Convert date/time columns to datetime and extract features
df['date'] = pd.to_datetime(df['date'])
df['year'] = df['date'].dt.year
df['month'] = df['date'].dt.month
df['day'] = df['date'].dt.day
df = df.drop(columns=['date'])  # Drop the original date column

# Encode categorical columns
for col in ['comp', 'round', 'day', 'venue', 'opponent', 'captain', 'formation', 'referee', 'match report', 'team']:
    df[col] = LabelEncoder().fit_transform(df[col])


In [ ]:
df = df.drop(columns=['notes'])

In [ ]:
train_df , val_df = train_test_split(df , test_size=0.2)
train_df , test_df =  train_test_split(train_df,test_size=0.2)
len(train_df),len(val_df),len(test_df)

In [ ]:
y = train_df['result']
X = train_df.drop(columns=['result'])
X

In [ ]:
xgb_selection = xgb.XGBClassifier(
    n_estimators=200, 
    max_depth=5, 
    learning_rate=0.05, 
    colsample_bytree=0.8, 
    subsample=0.8, 
    tree_method="hist", 
    enable_categorical=True
)
X.pop('time')
print(X.dtypes)  # Verify all columns are numeric or category

# Xgboost does not natively support categorical variables or text data without preprocessing (hence 4 cells above this cell is used)
xgb_selection.fit(X, y)  # Fit the model


IMPORTANT

In [ ]:

importances = xgb_selection.feature_importances_

# Set a threshold and select features
threshold = 0.01
selected_features = X.columns[importances > threshold]
X_selected = X[selected_features]

In [ ]:
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=200,max_depth=5,min_samples_split=5,random_state=1)
model.fit(X_selected,y)

In [ ]:
from sklearn.metrics import accuracy_score
X_val = val_df[selected_features] # important training df cols and val_df cols must be the same
ans = model.predict(X_val)
acc = accuracy_score(ans,val_df['result'])
print("Accuracy score:",acc*100)
ans

In [ ]:
X_test = test_df[selected_features]
result = model.predict(X_test)
acc = accuracy_score(result,test_df['result'])
print("Accuracy score:",acc*100)
result